<a href="https://colab.research.google.com/github/Nsharma96/Real-time-FER/blob/master/FER_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys,os
bpath = '/content/drive/My Drive/Data set' #datasetdir
sys.path.insert(0,bpath)
os.chdir(bpath)
mpath = '.models/' #models folder

In [4]:
dataset = pd.read_csv('./fer2013/fer2013.csv')
# dataset = pd.read_csv('fer2013.csv')
display(dataset.head())

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
print("\n****************************************************************************************\n")
print("Number of traing and test examples")
display(dataset['Usage'].value_counts())
print("\n****************************************************************************************\n")
print("Distribution of emotions")
display(dataset['emotion'].value_counts())


****************************************************************************************

Number of traing and test examples


Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64


****************************************************************************************

Distribution of emotions


3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: emotion, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers,Sequential,optimizers
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,TensorBoard 
from tensorflow.keras.regularizers import l2

In [0]:
def buildModel(learning_rate):
  # 28709, 2304
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48,1), kernel_regularizer=l2(0.01)))
  model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
  model.add(Dropout(0.5))

  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.5))    

  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.5))    

  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())

  model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.5))    

  model.add(Flatten())
  
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))

  model.add(Dense(7, activation='softmax'))

  adam = optimizers.Adam(lr = learning_rate)
  model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])    
  print(model.summary())
  return model

  #Model architecture https://raw.githubusercontent.com/Paulymorphous/Facial-Emotion-Recognition/master/Emotion_Recognition(CNN)_FER2013.ipynb

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint(mpath, monitor='val_loss', verbose=1, save_best_only=True)

In [9]:
# data["Team"]= data["Team"].str.split("t", n = 1, expand = True) 
dataset["pixels"] = dataset["pixels"].apply(lambda x: x.split(" "))
dataset["pixels"] = dataset["pixels"].apply(lambda x: [float(k)/255 for k in x])
# dataset["pixels"] = dataset["pixels"].apply(lambda x: (x.reshape(48,48).astype(np.float32))/255)
  
display(dataset.head())
dataset.dropna(inplace=True)

,emotion,pixels,Usage
0,0,"[0.27450980392156865, 0.3137254901960784, 0.32...",Training
1,0,"[0.592156862745098, 0.5882352941176471, 0.5764...",Training
2,2,"[0.9058823529411765, 0.8313725490196079, 0.611...",Training
3,4,"[0.09411764705882353, 0.12549019607843137, 0.1...",Training
4,6,"[0.01568627450980392, 0.0, 0.0, 0.0, 0.0, 0.0,...",Training


In [0]:
data_train = dataset[dataset["Usage"]=="Training"]
data_test_public = dataset[dataset["Usage"]=="PublicTest"]

In [0]:
x_train = []
for i in range(len(data_train["pixels"])):
  x_train.append(np.reshape(data_train["pixels"].iloc[i],(48,48)))

In [0]:
y_train = data_train["emotion"]
y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)

In [13]:
# print(x_train[0].shape)
# k = np.expand_dims(data_train["pixels"][0],-1)
# # print(data_train["pixels"][0].shape)
np.array(x_train).shape
# print(k.shape)

(28709, 48, 48)

In [14]:
epoch = 100
batch_size = 64
learning_rate = 0.001
model = buildModel(learning_rate)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       5

In [0]:
model.fit(np.expand_dims(np.array(x_train),-1),y_train,epochs = epoch,batch_size = batch_size,validation_split = 0.2,shuffle = True,callbacks=[lr_reducer, checkpointer],verbose=1)

In [0]:
fer_json = model.to_json()
with open("./fer2013/fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("./fer2013/fer.h5")